In [1]:
# import cv2
import pandas as pd
import numpy as np
import glob,os

In [2]:
image_pathes='archive/*'
patient_list = list(glob.glob(image_pathes))
print('Number of the patient :'+' '+str(len(patient_list)))
# print(patient_list)

Number of the patient : 280


In [3]:
# total images in both cancerous and non-cancerous cell
sum = 0
for i in range(len(patient_list)):
    sum+=len(glob.glob(patient_list[i]+'/*/*'))
print(sum)

278082


In [4]:
# loading all the image locations 
# patient_list(archive/patient_class)/0 or 1()/image
all_image=[]
for i in range(len(patient_list)):
#     print(patient_list[i])
#     print(glob.glob(patient_list[i]+'/*/*'))
    if i==279:
        print(glob.glob(patient_list[i]+'/*/*'))
    all_image.append(glob.glob(patient_list[i]+'/*/*'))

print(len(all_image))
# print(all_image)

['archive\\IDC_regular_ps50_idx5\\10253\\0', 'archive\\IDC_regular_ps50_idx5\\10253\\1', 'archive\\IDC_regular_ps50_idx5\\10254\\0', 'archive\\IDC_regular_ps50_idx5\\10254\\1', 'archive\\IDC_regular_ps50_idx5\\10255\\0', 'archive\\IDC_regular_ps50_idx5\\10255\\1', 'archive\\IDC_regular_ps50_idx5\\10256\\0', 'archive\\IDC_regular_ps50_idx5\\10256\\1', 'archive\\IDC_regular_ps50_idx5\\10257\\0', 'archive\\IDC_regular_ps50_idx5\\10257\\1', 'archive\\IDC_regular_ps50_idx5\\10258\\0', 'archive\\IDC_regular_ps50_idx5\\10258\\1', 'archive\\IDC_regular_ps50_idx5\\10259\\0', 'archive\\IDC_regular_ps50_idx5\\10259\\1', 'archive\\IDC_regular_ps50_idx5\\10260\\0', 'archive\\IDC_regular_ps50_idx5\\10260\\1', 'archive\\IDC_regular_ps50_idx5\\10261\\0', 'archive\\IDC_regular_ps50_idx5\\10261\\1', 'archive\\IDC_regular_ps50_idx5\\10262\\0', 'archive\\IDC_regular_ps50_idx5\\10262\\1', 'archive\\IDC_regular_ps50_idx5\\10264\\0', 'archive\\IDC_regular_ps50_idx5\\10264\\1', 'archive\\IDC_regular_ps50_idx5

In [5]:
print(all_image)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
# sample
import skimage
from skimage.io import imread , imread_collection 
import matplotlib.pyplot as plt
# print((all_image[0]))
img_sample = imread(all_image[0][0])
plt.title('patienID:'
          +str(all_image[2][0].split('/')[1])+'\n'
         'class:'+str(all_image[2][0].split('/')[2]))
plt.imshow(img_sample)

IndexError: list index out of range

In [ ]:
# creating dataframe
patient_id=[all_image[i][j].split('/')[1] for i in range(0,len(all_image)) for j in range(0,len(all_image[i]))]
# print(patient_id)
class_test=[all_image[i][j].split('/')[2] for i in range(0,len(all_image)) for j in range(0,len(all_image[i]))]
# print(class_test)
patient_pathes = [all_image[i][x]for i in range(0,len(all_image))for x in range(0,len(all_image[i]))]


In [ ]:
df_original=pd.DataFrame({
    'Patient ID':patient_id,
    'Diagnosis':class_test,
    'Pathes':patient_pathes
})

In [ ]:
df_original.to_csv('Original_data.csv')

In [ ]:
df_original=pd.read_csv('Original_data.csv')
df_original.head(n=1)

In [ ]:
df_original=df_original.drop(labels='Unnamed: 0',axis=1)
df_original.head(n=1)

<b>train test split</b>

In [ ]:
from sklearn.model_selection import train_test_split
df_train,df_test=train_test_split(df_original,test_size=0.4,)

In [ ]:
df_train.to_csv('train_data.csv')
df_test.to_csv('test_data.csv')

In [ ]:
df_train=pd.read_csv('train_data.csv')
print(len(df_train))
df_train.head(n=1)

In [ ]:
# df_train=df_train.drop(labels='Unnamed: 0.1',axis=1)
df_train=df_train.drop(labels='Unnamed: 0',axis=1)
df_train.head(n=1)

<b>random sampling from train</b>

<p style="color:yellow">preloaded random samples from train</p>

In [ ]:
# skip this for the first time
random_train=pd.read_csv('random_train.csv')
random_train.head(n=1)
# random_train=random_train.drop( labels='Unnamed: 0',axis=1)

<p style="color:yellow">new random sample generation</p>

In [ ]:
import random
# train_index=pd.r
random_index=random.sample(range(0,len(df_train)),10000)
random_train=df_train.loc[random_index]
print(len(random_train))
random_train.head(n=1)

In [ ]:
# random_train['Pixels']=random_train=random_train.drop( labels='Unnamed: 0',axis=1)
# random_train['Pixels']=random_train
# random_train=random_train.drop( labels='Unnamed: 0.1',axis=1)

In [ ]:
random_train=random_train.drop( labels='Unnamed: 0',axis=1)

In [ ]:
# original image shape (50,50,3)
from PIL.Image import open
# random_train['Pixels']=np.array(random_train['Pathes'].map(lambda x:np.asarray(open(x).resize((128,128)))))
random_train['Pixels']=random_train['Pathes'].map(lambda x:np.asarray(open(x).resize((128,128))))

In [ ]:
random_train.to_csv('random_train.csv')
random_train.head(n=6)

In [ ]:
bm_quantity=random_train['Diagnosis'].value_counts()
print(bm_quantity)
bm_quantity.plot.bar(
  color=['g','r']
)

In [ ]:
X_train=random_train['Pixels']
y=np.array(random_train['Diagnosis'])

In [ ]:
print(type(y))
print(y)

<b>to balance the distribution at first flatten the image, then smote, then back to original shape</b>

In [ ]:
# training set
# preprocessing dataset
from keras.preprocessing.image import img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
img_list = []
for i in range (0,len(X_train)):
    brain_img =np.array(X_train)[i]
    img_arr = img_to_array(brain_img).astype(np.float32)
    img_list.append(preprocess_input(img_arr))
X_scaled = np.array(img_list)

print(X_scaled.shape)

In [ ]:
X_train_shape = X_scaled.shape[1]*X_scaled.shape[2]*X_scaled.shape[3]
X_Flatten = X_scaled.reshape(X_scaled.shape[0],X_train_shape)

In [ ]:
# start resampling via smote
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0)
x_sm,y_sm = sm.fit_sample(X_Flatten,y)

In [ ]:
print('The size of our dataset after oversampling:'+' '+str(y_sm.shape[0]))

In [ ]:
import seaborn as sns
sns.countplot(y_sm)

In [ ]:
row,col,channel = 128 ,128 , 3 
X_sm_reshaped = x_sm.reshape(len(x_sm),row,col,channel)
# chec the shape of our training set
print('The shape of the training set after reshape it :'+" "+str(X_sm_reshaped.shape))

<b>model</b>

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , GlobalAveragePooling2D
from keras.applications import MobileNetV2
num_classes = 1
model = Sequential([
     MobileNetV2(input_shape=(224,224,3),weights="imagenet",include_top=False), 
     GlobalAveragePooling2D(),
     Dense(num_classes, activation='sigmoid',name='preds'),
])
model.layers[0].trainable= False
# show model summary
model.summary()

In [ ]:
import keras
model.compile(optimizer='adam',
             loss=keras.losses.binary_crossentropy,
             metrics=['accuracy'])
model.fit(X_sm_reshaped,y_sm
          ,epochs=20,verbose=1)

In [ ]:
df_test=pd.read_csv('test_data.csv')
df_test.head(n=1)

In [ ]:
df_test=df_test.drop(labels='Unnamed: 0',axis=1)
df_test.head(n=1)

In [ ]:
import random
# train_index=pd.r
random_index=random.sample(range(0,len(df_test)),2000)
random_test=df_test.loc[random_index]
print(len(random_test))
random_test.head(n=1)

In [ ]:
random_test=random_test.drop( labels='Pixels',axis=1)
random_test.head(n=1)

In [ ]:
# from PIL.Image import open
# random_train['Pixels']=np.array(random_train['Pathes'].map(lambda x:np.asarray(open(x).resize((128,128)))))
random_test['Pixels']=random_test['Pathes'].map(lambda x:np.asarray(open(x).resize((128,128))))

In [ ]:
X_test=random_test['Pixels']
y_test=np.array(random_test['Diagnosis'])

In [ ]:
img_list = []
for i in range (0,len(X_test)):
    brain_img =np.array(X_test)[i]
    img_arr = img_to_array(brain_img).astype(np.float32)
    img_list.append(preprocess_input(img_arr))
X_scaled_test = np.array(img_list)
print(X_scaled_test.shape)

In [ ]:
eval_score = model.evaluate(X_scaled_test,y_test)
print('The loss of the evaluation :'+' '+str(eval_score[0]*100))
print('The accuracy of the evaluation :'+' '+str(eval_score[1]*100))

In [ ]:
from sklearn.metrics import classification_report , confusion_matrix
y_pred = model.predict_classes(X_scaled_test)
print(y_pred)